In [1]:
from pymodbus.client.sync import ModbusTcpClient as ModbusClient
# from pymodbus.client.sync import ModbusUdpClient as ModbusClient
# from pymodbus.client.sync import ModbusSerialClient as ModbusClient
# from pymodbus.client.sync import ModbusTcpClient
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadBuilder
from pymodbus.payload import BinaryPayloadDecoder
from collections import OrderedDict
import struct


In [ ]:
client = ModbusClient('192.168.2.16')
client.connect()

In [ ]:
con

In [ ]:
result.registers

In [27]:
from pymodbus.client.sync import ModbusTcpClient as ModbusClient

import logging
FORMAT = ('%(asctime)-15s %(threadName)-15s '
          '%(levelname)-8s %(module)-15s:%(lineno)-8s %(message)s')
logging.basicConfig(format=FORMAT)
log = logging.getLogger()
log.setLevel(logging.DEBUG)

UNIT = 0x1


def run_sync_client():
    # ------------------------------------------------------------------------#

    client = ModbusClient('192.168.2.16', port=502)
    client.connect()
    
    log.debug("Write to a holding register and read back")
    builder = BinaryPayloadBuilder(byteorder=Endian.Big,
                                   wordorder=Endian.Big)
    builder.add_32bit_float(22.34546456546156516)
    payload = builder.build()
    
    
#     Can write registers
    registers = builder.to_registers()
#     rr = client.write_registers(24590, registers, unit=UNIT)

    # Or can write encoded binary string
#     print(payload)
#     rr = client.write_register(24590, payload, skip_encode=True, unit=UNIT)
#     print(rr)
#     assert(rr.registers[0] == 10)       # test the expected value

#     log.debug("Read input registers")
    rr = client.read_input_registers(8210, 2, unit=UNIT)
# #     print(rr)
# #     arguments = {
# #         'read_address':    1,
# #         'read_count':      8,
# #         'write_address':   1,
# #         'write_registers': [20]*8,
# #     }
# #     log.debug("Read write registeres simulataneously")
# #     rq = client.readwrite_registers(unit=UNIT, **arguments)
    rr = client.read_holding_registers(24594, 2, unit=UNIT)
#     print(rr)
#     assert(not rq.isError())     # test that we are not an error
#     assert(rq.registers == [20]*8)      # test the expected value
#     assert(rr.registers == [20]*8)      # test the expected value

    # ----------------------------------------------------------------------- #
    # close the client
    # ----------------------------------------------------------------------- #
    client.close()
    return rr


rr = run_sync_client()

2018-12-08 03:19:08,822 MainThread      DEBUG    <ipython-input-27-435ed2f7b186>:19       Write to a holding register and read back
2018-12-08 03:19:08,822 MainThread      DEBUG    payload        :124      [16818, 50051]
2018-12-08 03:19:08,823 MainThread      DEBUG    transaction    :111      Current transaction state - IDLE
2018-12-08 03:19:08,823 MainThread      DEBUG    transaction    :115      Running transaction 1
2018-12-08 03:19:08,823 MainThread      DEBUG    transaction    :206      SEND: 0x0 0x1 0x0 0x0 0x0 0x6 0x1 0x4 0x20 0x12 0x0 0x2
2018-12-08 03:19:08,824 MainThread      DEBUG    sync           :75       New Transaction state 'SENDING'
2018-12-08 03:19:08,824 MainThread      DEBUG    transaction    :209      Changing transaction state from 'SENDING' to 'WAITING FOR REPLY'
2018-12-08 03:19:08,827 MainThread      DEBUG    transaction    :284      Changing transaction state from 'WAITING FOR REPLY' to 'PROCESSING REPLY'
2018-12-08 03:19:08,828 MainThread      DEBUG    tran

In [28]:
rr

In [29]:
decoder = BinaryPayloadDecoder.fromRegisters(rr.registers, byteorder=Endian.Big, wordorder=Endian.Big)


2018-12-08 03:19:10,994 MainThread      DEBUG    payload        :299      [18534, 1260]


In [30]:
decoder.decode_32bit_float()

2018-12-08 03:19:11,661 MainThread      DEBUG    payload        :343      [b'Hf', b'\x04\xec']


235539.6875

In [6]:
bit = decoder.decode_32bit_int()
print(bit)

2018-12-08 03:12:45,810 MainThread      DEBUG    payload        :343      [b'A\xb2', b'\xc3\x83']


1102234499


In [7]:
x = struct.pack('i', bit)
result = struct.unpack('f', x)

In [8]:
result[0] * 57

1273.6914882659912

In [9]:
builder = BinaryPayloadBuilder(byteorder=Endian.Big,
                                   wordorder=Endian.Little)

In [ ]:
builder.add_32bit_float(22.34)
payload = builder.build()
payload

In [ ]:
#!/usr/bin/env python
"""
Pymodbus Payload Building/Decoding Example
--------------------------------------------------------------------------
# Run modbus-payload-server.py or synchronous-server.py to check the behavior
"""
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.payload import BinaryPayloadBuilder
from pymodbus.client.sync import ModbusTcpClient as ModbusClient
from pymodbus.compat import iteritems
from collections import OrderedDict
# --------------------------------------------------------------------------- #
# configure the client logging
# --------------------------------------------------------------------------- #
import logging
FORMAT = ('%(asctime)-15s %(threadName)-15s'
          ' %(levelname)-8s %(module)-15s:%(lineno)-8s %(message)s')
logging.basicConfig(format=FORMAT)
log = logging.getLogger()
log.setLevel(logging.INFO)
def run_binary_payload_ex():
    # ----------------------------------------------------------------------- #
    # We are going to use a simple client to send our requests
    # ----------------------------------------------------------------------- #
    client = ModbusClient('192.168.2.16')
    client.connect()

    address = 8200
    result = client.read_holding_registers(address, 10, unit=1)
    print("-" * 60)
    print("Registers")
    print("-" * 60)
    print(result.registers)
    print("\n")
    decoder = BinaryPayloadDecoder.fromRegisters(result.registers, byteorder=Endian.Little, 
                                                 wordorder=Endian.Little)
    decoded = OrderedDict([
        ('string', decoder.decode_string(8)),
        ('bits', decoder.decode_bits()),
        ('8int', decoder.decode_8bit_int()),
        ('8uint', decoder.decode_8bit_uint()),
        ('16int', decoder.decode_16bit_int()),
        ('16uint', decoder.decode_16bit_uint()),
        ('32int', decoder.decode_32bit_int()),
        ('ignore', decoder.skip_bytes(8)),
    ])
    print("-" * 60)
    print("Decoded Data")
    print("-" * 60)
    for name, value in iteritems(decoded):
        print("%s\t" % name, value if isinstance(value, int) else value)
    for name, value in iteritems(decoded):
        print("%s\t" % name, hex(value) if isinstance(value, int) else value)

    
    # ----------------------------------------------------------------------- #
    # close the client
    # ----------------------------------------------------------------------- #
    client.close()
    return decoded

In [ ]:
# import the various server implementations
# --------------------------------------------------------------------------- #
# from pymodbus.client.sync import ModbusTcpClient as ModbusClient
from pymodbus.client.sync import ModbusUdpClient as ModbusClient
# from pymodbus.client.sync import ModbusSerialClient as ModbusClient

# --------------------------------------------------------------------------- #
# configure the client logging
# --------------------------------------------------------------------------- #
import logging
FORMAT = ('%(asctime)-15s %(threadName)-15s '
          '%(levelname)-8s %(module)-15s:%(lineno)-8s %(message)s')
logging.basicConfig(format=FORMAT)
log = logging.getLogger()
log.setLevel(logging.DEBUG)

UNIT = 0x1


def run_sync_client():
    # ------------------------------------------------------------------------#
    # choose the client you want
    # ------------------------------------------------------------------------#
    # make sure to start an implementation to hit against. For this
    # you can use an existing device, the reference implementation in the tools
    # directory, or start a pymodbus server.
    #
    # If you use the UDP or TCP clients, you can override the framer being used
    # to use a custom implementation (say RTU over TCP). By default they use
    # the socket framer::
    #
    #    client = ModbusClient('localhost', port=5020, framer=ModbusRtuFramer)
    #
    # It should be noted that you can supply an ipv4 or an ipv6 host address
    # for both the UDP and TCP clients.
    #
    # There are also other options that can be set on the client that controls
    # how transactions are performed. The current ones are:
    #
    # * retries - Specify how many retries to allow per transaction (default=3)
    # * retry_on_empty - Is an empty response a retry (default = False)
    # * source_address - Specifies the TCP source address to bind to
    #
    # Here is an example of using these options::
    #
    #    client = ModbusClient('localhost', retries=3, retry_on_empty=True)
    # ------------------------------------------------------------------------#
    client = ModbusClient('192.168.2.1', port=502)

    client.connect()

    # ------------------------------------------------------------------------#
    # specify slave to query
    # ------------------------------------------------------------------------#
    # The slave to query is specified in an optional parameter for each
    # individual request. This can be done by specifying the `unit` parameter
    # which defaults to `0x00`
    # ----------------------------------------------------------------------- #
    log.debug("Reading Coils")
    rr = client.read_coils(16396, 1, unit=UNIT)
    log.debug(rr)


    # ----------------------------------------------------------------------- #
    # example requests
    # ----------------------------------------------------------------------- #
    # simply call the methods that you would like to use. An example session
    # is displayed below along with some assert checks. Note that some modbus
    # implementations differentiate holding/input discrete/coils and as such
    # you will not be able to write to these, therefore the starting values
    # are not known to these tests. Furthermore, some use the same memory
    # blocks for the two sets, so a change to one is a change to the other.
    # Keep both of these cases in mind when testing as the following will
    # _only_ pass with the supplied async modbus server (script supplied).
    # ----------------------------------------------------------------------- #
#     log.debug("Write to a Coil and read back")
#     rq = client.write_coil(0, True, unit=UNIT)
#     rr = client.read_coils(0, 1, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error
#     assert(rr.bits[0] == True)          # test the expected value

#     log.debug("Write to multiple coils and read back- test 1")
#     rq = client.write_coils(1, [True]*8, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error
#     rr = client.read_coils(1, 21, unit=UNIT)
#     assert(not rr.isError())     # test that we are not an error
#     resp = [True]*21

#     # If the returned output quantity is not a multiple of eight,
#     # the remaining bits in the final data byte will be padded with zeros
#     # (toward the high order end of the byte).

#     resp.extend([False]*3)
#     assert(rr.bits == resp)         # test the expected value

#     log.debug("Write to multiple coils and read back - test 2")
#     rq = client.write_coils(1, [False]*8, unit=UNIT)
#     rr = client.read_coils(1, 8, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error
#     assert(rr.bits == [False]*8)         # test the expected value

#     log.debug("Read discrete inputs")
#     rr = client.read_discrete_inputs(0, 8, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error

#     log.debug("Write to a holding register and read back")
# #     rq = client.write_register(1, 10, unit=UNIT)
#     rr = client.read_holding_registers(24578, 1, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error
#     assert(rr.registers[0] == 10)       # test the expected value

# #     log.debug("Write to multiple holding registers and read back")
# #     rq = client.write_registers(1, [10]*8, unit=UNIT)
# #     rr = client.read_holding_registers(1, 8, unit=UNIT)
# #     assert(not rq.isError())     # test that we are not an error
# #     assert(rr.registers == [10]*8)      # test the expected value

#     log.debug("Read input registers")
#     rr = client.read_input_registers(8196, 8, unit=UNIT)
#     assert(not rq.isError())     # test that we are not an error

#     arguments = {
#         'read_address':    1,
#         'read_count':      8,
#         'write_address':   1,
#         'write_registers': [20]*8,
#     }
# #     log.debug("Read write registeres simulataneously")
# #     rq = client.readwrite_registers(unit=UNIT, **arguments)
# #     rr = client.read_holding_registers(1, 8, unit=UNIT)
# #     assert(not rq.isError())     # test that we are not an error
# #     assert(rq.registers == [20]*8)      # test the expected value
# #     assert(rr.registers == [20]*8)      # test the expected value

#     # ----------------------------------------------------------------------- #
#     # close the client
#     # ----------------------------------------------------------------------- #
    client.close()



run_sync_client()

In [ ]:
client = ModbusClient('192.168.2.16', port=502)

In [ ]:
client.connect()

In [ ]:
log.debug("Read input registers")
rr = client.read_input_registers(8196, 8, unit=UNIT)
assert(not rq.isError())     # test that we are not an error